# PostgreSQL

For small sizes of datasets, for example, products, product categories, users, and order transactions, we can use SQL with high consistency of data (ACID for transactions). Cassandra does not support ACID but supports eventual consistency.

In case when a user modifies an item in the order, we must update the order in all of the replication servers. So that the seller can have update-to-date information on the order and pack exactly the items that the user wants.

## Setup 

### PostgreSQL - Single Node

First, pull the official image from the Docker Hub:

```bash
$ docker pull postgres
```

To run a single node, let's create a container from the downloaded image:

```bash
$ docker run --name purchase_db \
-v E:/shared_data/postgres:/var/lib/postgresql/data \
-e POSTGRES_USER=postgres \
-e POSTGRES_PASSWORD=mysecretpassword  \
-p 5432:5432 -d postgres
```

To open `qsql`

```bash
docker exec -it purchase_db bash
root@d59f357ea8b7:/# psql -U postgres
```


### PostgreSQL Cluster

The most convenient way to run a cluster is to run the image provided by VMware.

1. Download the image
```bash
$ docker pull bitnami/postgresql
Using default tag: latest
latest: Pulling from bitnami/postgresql
0de4c22220bb: Pull complete
Digest: sha256:100f658b4e69b7fb1a87f0379f52bc6aec84813d91d8a4e953774c5edd4658df
Status: Downloaded newer image for bitnami/postgresql:latest
docker.io/bitnami/postgresql:latest
```


2. Use `docker-compose.yml` to configure
Download the template file for `docker-compose.yml` from
 https://raw.githubusercontent.com/bitnami/containers/main/bitnami/postgresql/docker-compose.yml

Here are my configurations

```
version: '2'

networks:
  purchase_db-network:
    driver: bridge
    
services:
  purchase_db-master:
    image: 'bitnami/postgresql:latest'
    networks:
      - purchase_db-network
    ports:
      - '5433:5432'
    volumes:
      - 'postgresql_master_data:/bitnami/postgresql'
    environment:
      - POSTGRESQL_DATABASE=ecommerce
      - POSTGRESQL_REPLICATION_MODE=master
      - POSTGRESQL_REPLICATION_USER=repl_user
      - POSTGRESQL_REPLICATION_PASSWORD=repl_password
      - POSTGRESQL_USERNAME=postgres
      - POSTGRESQL_PASSWORD=mysecretpassword
  purchase_db-slave:
    image: 'bitnami/postgresql:latest'
    networks:
      - purchase_db-network
    ports:
      - '5432'
    depends_on:
      - purchase_db-master
    environment:
      - POSTGRESQL_REPLICATION_MODE=slave
      - POSTGRESQL_REPLICATION_USER=repl_user
      - POSTGRESQL_REPLICATION_PASSWORD=repl_password
      - POSTGRESQL_MASTER_HOST=purchase_db-master
      - POSTGRESQL_PASSWORD=mysecretpassword2
      - POSTGRESQL_MASTER_PORT_NUMBER=5432

volumes:
  postgresql_master_data:
```



3. Run the cluster

We can scale the cluster easily with the `--scale` option. Here, we create 2 replicas and one master.

```bash
$ docker-compose up --detach --scale purchase_db-master=1 --scale purchase_db-slave=2
```

## Working with Data

In [1]:
import numpy as np
from pathlib import Path
import time

from datatools import DBWrapper, LoadDataToPostgre

In [2]:
# CSV info and configuration
files = ["2019-Dec.csv"]  # ["2019-Oct.csv", "2019-Nov.csv", "2019-Dec.csv"]
path = "E:/coding/input/ecommerce-behavior-data-from-multi-category-store/"
DTYPE = {
    "event_time": str,
    "event_type": "category",
    "product_id": str,
    "category_id": str,
    "category_code": str,
    "brand": str,
    "price": np.float16,
    "user_id": str,
    "user_session": str,
}

## Postgre Database

`psycopg2` creates a new transaction for the first time using `cursor`. All statements are in the same transaction until the `commit` or `rollback` is called.

* `commit`: commits the transaction permanently. We have to confirm unless `autocommit` is set. Otherwise, the data might not be saved. 
* `rollback`: when something goes wrong, the transaction is rolled back.

Without `commit` or `rollback`, the database might be bloated or locked. Therefore, do not leave the connection open for a long time.

Make sure to create a database in PostgreSQL first. In this module, the database is `ecommerce`:

In bash:
```bash
$ echo "SELECT 'CREATE DATABASE ecommerce' WHERE NOT EXISTS (SELECT FROM pg_database WHERE datname = 'ecommerce')\gexec" | psql
```

Or SQL query
```
SELECT 'CREATE DATABASE ecommerce'
WHERE NOT EXISTS (SELECT FROM pg_database WHERE datname = 'ecommerce')\gexec
```

In [3]:
table_name = {
    "table_category": "table_category",
    "table_product": "table_product",
    "table_user": "table_user",
    "table_order": "table_order",
}
info = {
    "database": "ecommerce",
    "user": "postgres",
    "password": "mysecretpassword",
    "host": "192.168.137.101",
    "port": "5433",
    "table_name": table_name,
}

postgres_conn = DBWrapper("postgres", info)

In [4]:
BATCH_SIZE = 10000
CHUNK_SIZE = 10**6
# if the program stop in the middle of the process, set SKIP_ROWS to continue
SKIP_ROWS = None  # None
loader = LoadDataToPostgre()

for name in files:
    file_path = Path(path + name)
    print(file_path)
    start = time.time()
    loader.save_to_database(
        postgres_conn,
        file_path,
        dtype=DTYPE,
        BATCH_SIZE=BATCH_SIZE,
        CHUNK_SIZE=CHUNK_SIZE,
        SKIP_ROWS=SKIP_ROWS,
    )
    print("Running time = ", time.time() - start)

DEBUG:datatools: Start loading data from CSV file


E:\coding\input\ecommerce-behavior-data-from-multi-category-store\2019-Dec.csv


DEBUG:datatools:Start inserting data to Database
DEBUG:datatools: Finished inserting into Database


Running time =  688.3083052635193


### Close the Connection

In [5]:
postgres_conn.disconnect()

### Test

In [6]:
postgres_conn = DBWrapper("postgres", info)

In [7]:
postgres_conn.execute("SELECT count(*) FROM table_category")
postgres_conn.fetchall()

[(1162,)]

In [8]:
postgres_conn.execute("SELECT count(*) FROM table_order")
postgres_conn.fetchall()

[(970005,)]

In [9]:
postgres_conn.execute("SELECT count(*) FROM table_user")
postgres_conn.fetchall()

[(4577232,)]